In [2]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.1,
)

In [4]:
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=80,
    memory_key="chat_history",
    return_messages=True,  # output - for chat
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI talking to a Human"),
        MessagesPlaceholder(
            variable_name="chat_history"
        ),  # unknown sender(human/ai/system), unlimited messages
        ("human", "{question}"),
    ]
)

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=prompt,
    verbose=True,
)

In [5]:
chain.predict(question="My name is Jimin")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a Human
Human: My name is Jimin

> Finished chain.


'Hello Jimin! How can I assist you today?'

In [6]:
chain.predict(question="I like Dogs, especially Korean Jindo.")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a Human
Human: My name is Jimin
AI: Hello Jimin! How can I assist you today?
Human: I like Dogs, especially Korean Jindo.

> Finished chain.


"That's great to hear, Jimin! Korean Jindos are known for their loyalty and intelligence. Do you have a Korean Jindo as a pet or are you thinking of getting one?"

In [7]:
chain.predict(question="What is my name?")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a Human
System: The human introduces themselves as Jimin.
AI: Hello Jimin! How can I assist you today?
Human: I like Dogs, especially Korean Jindo.
AI: That's great to hear, Jimin! Korean Jindos are known for their loyalty and intelligence. Do you have a Korean Jindo as a pet or are you thinking of getting one?
Human: What is my name?

> Finished chain.


'Your name is Jimin.'